In [8]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import os
import praw

In [10]:
# Define subreddits and keywords
subreddits = ["iPhone16", "iPhone16Pro", "iPhone16ProMax"]
keywords = ["camera", "battery", "design", "display", "performance", "durability"]

In [12]:
# Define the release date of iPhone 16 Pro
release_date = datetime(2024, 9, 20, tzinfo=timezone.utc)  # September 20, 2024, in UTC
start_date = int(release_date.timestamp())  # Convert to UNIX timestamp
end_date = int(datetime.now(timezone.utc).timestamp())  # Current timestam

In [71]:
# Correct release date for iPhone 16 Pro
release_date = datetime(2024, 9, 20, tzinfo=timezone.utc)  # September 20, 2024, in UTC
start_date = int(release_date.timestamp())  # Convert to UNIX timestamp
end_date = int(datetime.now(timezone.utc).timestamp())  # Current timestamp

# Print the timestamps for debugging
print(f"Start Date (Unix): {start_date} -> {datetime.utcfromtimestamp(start_date)}")
print(f"End Date (Unix): {end_date} -> {datetime.utcfromtimestamp(end_date)}")

Start Date (Unix): 1726790400 -> 2024-09-20 00:00:00
End Date (Unix): 1738858389 -> 2025-02-06 16:13:09


/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/496210793.py:7: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f"Start Date (Unix): {start_date} -> {datetime.utcfromtimestamp(start_date)}")
/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/496210793.py:8: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f"End Date (Unix): {end_date} -> {datetime.utcfromtimestamp(end_date)}")


In [75]:
import praw
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv
import os
import time

# Load environment variables from .env file
load_dotenv()

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
)

# Function to fetch posts manually filtered by timestamps
def fetch_reddit_data(subreddit, keywords, start_date, end_date, limit=100):
    all_posts = []
    try:
        subreddit_obj = reddit.subreddit(subreddit)
        for submission in subreddit_obj.new(limit=limit):  # Fetch 'new' posts
            # Filter posts manually by timestamps
            if start_date <= submission.created_utc <= end_date:
                if any(keyword.lower() in submission.title.lower() or keyword.lower() in submission.selftext.lower() for keyword in keywords):
                    post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d')
                    all_posts.append({
                        "subreddit": subreddit,
                        "title": submission.title,
                        "selftext": submission.selftext,
                        "created_utc": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                        "post_date": post_date,
                        "net_votes": submission.score,
                        "num_comments": submission.num_comments,
                        "url": submission.url
                    })
            time.sleep(0.1)  # Add a short delay to avoid hitting rate limits
    except Exception as e:
        print(f"Error fetching data from r/{subreddit}: {e}")
    
    return all_posts

# Define the release date of iPhone 16 Pro
release_date = datetime(2024, 9, 20, tzinfo=timezone.utc)  # September 20, 2024
start_date = int(release_date.timestamp())  # Convert to UNIX timestamp
end_date = int(datetime.now(timezone.utc).timestamp())  # Current timestamp

# Debug: Print start and end dates
print(f"Fetching posts from {datetime.fromtimestamp(start_date, tz=timezone.utc)} to {datetime.fromtimestamp(end_date, tz=timezone.utc)}")

# Define subreddits and keywords
subreddits = ["iPhone16Pro", "iPhone16ProMax"]
keywords = ["camera", "battery", "design", "display", "performance", "durability"]

# Fetch data from multiple subreddits
all_posts = []
for subreddit in subreddits:
    print(f"Fetching data from subreddit: {subreddit}")
    posts = fetch_reddit_data(subreddit, keywords, start_date, end_date, limit=100)
    all_posts.extend(posts)

# Convert to a DataFrame
df = pd.DataFrame(all_posts)
print(df.head())  # Print the first few rows of the DataFrame

Fetching posts from 2024-09-20 00:00:00+00:00 to 2025-02-06 16:14:33+00:00
Fetching data from subreddit: iPhone16Pro


/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/1875748904.py:27: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d')
/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/1875748904.py:32: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "created_utc": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


Fetching data from subreddit: iPhone16ProMax
     subreddit                                   title  \
0  iPhone16Pro       Camera Button Slanted on Pro Max?   
1  iPhone16Pro    16Pro so lag! Just want my 13P back!   
2  iPhone16Pro                             Main camera   
3  iPhone16Pro                     iPhone 16 Pro Issue   
4  iPhone16Pro  Stock camera processes photos too much   

                                            selftext          created_utc  \
0  Got a 16 Pro Max last Saturday and I just noti...  2025-02-06 14:35:41   
1  My brand new 16Pro has been laggy/glitchy sinc...  2025-02-06 13:29:34   
2                                                     2025-02-06 09:39:05   
3  I’m not very happy with the drain on iPhone 16...  2025-02-06 08:05:01   
4        1st is stock camera, 2nd is Halide Mark ii   2025-02-06 08:01:51   

    post_date  net_votes  num_comments  \
0  2025-02-06          1             1   
1  2025-02-06          0            11   
2  2025-02-06    

In [79]:
# Save to a CSV file
df.to_csv("reddit_posts_iphone16pro.csv", index=False)
print("Data saved")


Data saved


In [77]:
num_rows = len(df)
print(f"Number of rows pulled: {num_rows}")

Number of rows pulled: 72


## Comments

In [91]:
import praw
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv
import os
import time

# Load environment variables from .env file
load_dotenv()

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
)

# Function to fetch posts and their comments
def fetch_reddit_data_with_comments(subreddit, keywords, start_date, end_date, post_limit=100, comment_limit=10):
    all_data = []
    try:
        subreddit_obj = reddit.subreddit(subreddit)
        for submission in subreddit_obj.new(limit=post_limit):  # Fetch 'new' posts
            # Filter posts by timestamps
            if start_date <= submission.created_utc <= end_date:
                if any(keyword.lower() in submission.title.lower() or keyword.lower() in submission.selftext.lower() for keyword in keywords):
                    submission.comments.replace_more(limit=None)  # Fetch all comments
                    comments = submission.comments.list()[:comment_limit]  # Limit the number of comments
                    for comment in comments:
                        all_data.append({
                            "subreddit": subreddit,
                            "post_title": submission.title,
                            "post_url": submission.url,
                            "post_created_utc": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            "comment_id": comment.id,
                            "comment_body": comment.body,
                            "comment_created_utc": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            "comment_net_votes": comment.score,
                        })
            time.sleep(0.1)  # Add a short delay to avoid hitting rate limits
    except Exception as e:
        print(f"Error fetching data from r/{subreddit}: {e}")
    
    return all_data

# Define the release date of iPhone 16 Pro
release_date = datetime(2024, 9, 20, tzinfo=timezone.utc)  # September 20, 2024
start_date = int(release_date.timestamp())  # Convert to UNIX timestamp
end_date = int(datetime.now(timezone.utc).timestamp())  # Current timestamp

# Debug: Print start and end dates
print(f"Fetching posts and comments from {datetime.fromtimestamp(start_date, tz=timezone.utc)} to {datetime.fromtimestamp(end_date, tz=timezone.utc)}")

# Define subreddits and keywords
subreddits = ["iPhone16Pro", "iPhone16ProMax"]
keywords = ["camera", "battery", "design", "display", "performance", "durability"]

# Fetch data from multiple subreddits
all_data = []
for subreddit in subreddits:
    print(f"Fetching posts and comments from subreddit: {subreddit}")
    data = fetch_reddit_data_with_comments(subreddit, keywords, start_date, end_date, post_limit=100, comment_limit=10)
    all_data.extend(data)

# Convert to a DataFrame
df_comments = pd.DataFrame(all_data)
print(df_comments.head())  # Print the first few rows of the DataFrame


Fetching posts and comments from 2024-09-20 00:00:00+00:00 to 2025-02-06 16:45:37+00:00
Fetching posts and comments from subreddit: iPhone16Pro


/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/2153276230.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "post_created_utc": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/2153276230.py:37: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "comment_created_utc": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


Fetching posts and comments from subreddit: iPhone16ProMax
     subreddit                            post_title  \
0  iPhone16Pro     Camera Button Slanted on Pro Max?   
1  iPhone16Pro     Camera Button Slanted on Pro Max?   
2  iPhone16Pro  16Pro so lag! Just want my 13P back!   
3  iPhone16Pro  16Pro so lag! Just want my 13P back!   
4  iPhone16Pro  16Pro so lag! Just want my 13P back!   

                                            post_url     post_created_utc  \
0  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 14:35:41   
1  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 14:35:41   
2  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 13:29:34   
3  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 13:29:34   
4  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 13:29:34   

  comment_id                                       comment_body  \
0    mbb93hn  Should be flush and even. Definitely not how y...   
1    mb

In [95]:
# Save to a CSV file
df_comments.to_csv("reddit_comments.csv", index=False)
print("Data saved to 'reddit_comments.csv'")

Data saved to 'reddit_comments.csv'


In [93]:
num_rows = len(df_comments)
print(f"Number of rows pulled: {num_rows}")

Number of rows pulled: 420


In [97]:
import praw
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv
import os
import time

# Load environment variables from .env file
load_dotenv()

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
)

# Function to fetch posts manually filtered by timestamps
def fetch_reddit_data(subreddit, keywords, start_date, end_date):
    all_posts = []
    try:
        subreddit_obj = reddit.subreddit(subreddit)
        # Use `None` to fetch all posts from the 'new' listing
        for submission in subreddit_obj.new(limit=None):  # No limit on posts
            # Filter posts manually by timestamps
            if start_date <= submission.created_utc <= end_date:
                if any(keyword.lower() in submission.title.lower() or keyword.lower() in submission.selftext.lower() for keyword in keywords):
                    post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d')
                    all_posts.append({
                        "subreddit": subreddit,
                        "title": submission.title,
                        "selftext": submission.selftext,
                        "created_utc": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                        "post_date": post_date,
                        "net_votes": submission.score,
                        "num_comments": submission.num_comments,
                        "url": submission.url
                    })
            time.sleep(0.5)  # Add a short delay between each post to avoid hitting rate limits
    except Exception as e:
        print(f"Error fetching data from r/{subreddit}: {e}")
    
    return all_posts

# Define the release date of iPhone 16 Pro
release_date = datetime(2024, 9, 20, tzinfo=timezone.utc)  # September 20, 2024
start_date = int(release_date.timestamp())  # Convert to UNIX timestamp
end_date = int(datetime.now(timezone.utc).timestamp())  # Current timestamp

# Debug: Print start and end dates
print(f"Fetching posts from {datetime.fromtimestamp(start_date, tz=timezone.utc)} to {datetime.fromtimestamp(end_date, tz=timezone.utc)}")

# Define subreddits and keywords
subreddits = ["iPhone16Pro", "iPhone16ProMax"]
keywords = ["camera", "battery", "design", "display", "performance", "durability"]

# Fetch data from multiple subreddits
all_posts = []
for subreddit in subreddits:
    print(f"Fetching data from subreddit: {subreddit}")
    posts = fetch_reddit_data(subreddit, keywords, start_date, end_date)
    all_posts.extend(posts)
    time.sleep(2)  # Add a delay between fetching different subreddits to avoid rate limits

# Convert to a DataFrame
df = pd.DataFrame(all_posts)
print(df.head())  # Print the first few rows of the DataFrame
print(len(df))

Fetching posts from 2024-09-20 00:00:00+00:00 to 2025-02-06 19:45:15+00:00
Fetching data from subreddit: iPhone16Pro


/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/4293807240.py:28: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d')
/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/4293807240.py:33: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "created_utc": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


Fetching data from subreddit: iPhone16ProMax
     subreddit                                   title  \
0  iPhone16Pro       Camera Button Slanted on Pro Max?   
1  iPhone16Pro    16Pro so lag! Just want my 13P back!   
2  iPhone16Pro                             Main camera   
3  iPhone16Pro                     iPhone 16 Pro Issue   
4  iPhone16Pro  Stock camera processes photos too much   

                                            selftext          created_utc  \
0  Got a 16 Pro Max last Saturday and I just noti...  2025-02-06 14:35:41   
1  My brand new 16Pro has been laggy/glitchy sinc...  2025-02-06 13:29:34   
2                                                     2025-02-06 09:39:05   
3  I’m not very happy with the drain on iPhone 16...  2025-02-06 08:05:01   
4        1st is stock camera, 2nd is Halide Mark ii   2025-02-06 08:01:51   

    post_date  net_votes  num_comments  \
0  2025-02-06          1             3   
1  2025-02-06          0            11   
2  2025-02-06    

In [101]:
# Save to a CSV file
df.to_csv("reddit_data_no_limit.csv", index=False)
print("Data saved to 'reddit_data_no_limit.csv'")

Data saved to 'reddit_data_no_limit.csv'


In [105]:
import praw
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv
import os
import time

# Load environment variables from .env file
load_dotenv()

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
)

# Function to fetch posts and their comments
def fetch_reddit_data_with_comments(subreddit, keywords, start_date, end_date, comment_limit=10):
    all_data = []
    try:
        subreddit_obj = reddit.subreddit(subreddit)
        # Fetch all available posts from the 'new' listing
        for submission in subreddit_obj.new(limit=None):  # No limit on posts
            # Filter posts manually by timestamps
            if start_date <= submission.created_utc <= end_date:
                if any(keyword.lower() in submission.title.lower() or keyword.lower() in submission.selftext.lower() for keyword in keywords):
                    try:
                        submission.comments.replace_more(limit=None)  # Fetch all comments
                        comments = submission.comments.list()[:comment_limit]  # Limit the number of comments
                        for comment in comments:
                            all_data.append({
                                "subreddit": subreddit,
                                "post_title": submission.title,
                                "post_url": submission.url,
                                "post_created_utc": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                "comment_id": comment.id,
                                "comment_body": comment.body,
                                "comment_created_utc": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                "comment_net_votes": comment.score,
                            })
                    except Exception as e:
                        print(f"Error fetching comments from post '{submission.title}': {e}")
            time.sleep(0.5)  # Add a short delay between processing posts
    except Exception as e:
        print(f"Error fetching data from r/{subreddit}: {e}")
    
    return all_data

# Define the release date of iPhone 16 Pro
release_date = datetime(2024, 9, 20, tzinfo=timezone.utc)  # September 20, 2024
start_date = int(release_date.timestamp())  # Convert to UNIX timestamp
end_date = int(datetime.now(timezone.utc).timestamp())  # Current timestamp

# Debug: Print start and end dates
print(f"Fetching posts and comments from {datetime.fromtimestamp(start_date, tz=timezone.utc)} to {datetime.fromtimestamp(end_date, tz=timezone.utc)}")

# Define subreddits and keywords
subreddits = ["iPhone16Pro", "iPhone16ProMax"]
keywords = ["camera", "battery", "design", "display", "performance", "durability"]

# Fetch data from multiple subreddits
all_data = []
for subreddit in subreddits:
    print(f"Fetching posts and comments from subreddit: {subreddit}")
    data = fetch_reddit_data_with_comments(subreddit, keywords, start_date, end_date, comment_limit=10)
    all_data.extend(data)
    time.sleep(2)  # Add a delay between fetching data from different subreddits to avoid API rate limits

# Convert to a DataFrame
df_comments = pd.DataFrame(all_data)
print(df_comments.head())  # Print the first few rows of the DataFrame
print(len(df_comments))

Fetching posts and comments from 2024-09-20 00:00:00+00:00 to 2025-02-06 20:32:03+00:00
Fetching posts and comments from subreddit: iPhone16Pro


/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/1859655507.py:36: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "post_created_utc": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
/var/folders/lb/69kfl84s4gs1jvnfcnp4ky740000gn/T/ipykernel_2582/1859655507.py:39: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "comment_created_utc": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


Fetching posts and comments from subreddit: iPhone16ProMax
     subreddit                            post_title  \
0  iPhone16Pro     Camera Button Slanted on Pro Max?   
1  iPhone16Pro     Camera Button Slanted on Pro Max?   
2  iPhone16Pro     Camera Button Slanted on Pro Max?   
3  iPhone16Pro  16Pro so lag! Just want my 13P back!   
4  iPhone16Pro  16Pro so lag! Just want my 13P back!   

                                            post_url     post_created_utc  \
0  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 14:35:41   
1  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 14:35:41   
2  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 14:35:41   
3  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 13:29:34   
4  https://www.reddit.com/r/iPhone16Pro/comments/...  2025-02-06 13:29:34   

  comment_id                                       comment_body  \
0    mbb93hn  Should be flush and even. Definitely not how y...   
1    mb

In [107]:
# Save to a CSV file
df_comments.to_csv("reddit_comments_no_post_limit.csv", index=False)
print("Data saved to 'reddit_comments_no_post_limit.csv'")

Data saved to 'reddit_comments_no_post_limit.csv'
